In [1]:
#importing python libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [2]:
import requests
import pandas as pd

#Call FRED API
# Define your API key
api_key = '2a2e9be69431487407cdf6e64bb58c72'

# Define the endpoint and parameters
endpoint = 'https://api.stlouisfed.org/fred/series/observations'
params = {'series_id': 'CUSR0000SEEB', 'api_key': api_key, 'file_type': 'json', 'observation_start': '2013-01-01'}

# Make the request
response = requests.get(endpoint, params=params)

# Parse the response
data = response.json()

# Create a pandas DataFrame
Fred_CC = pd.DataFrame(data['observations'])

# drop the first 2 columns
Fred_CC.drop(Fred_CC.columns[:3], axis=1, inplace=True)

# Convert the object 'value' to numeric
Fred_CC['value'] = pd.to_numeric(Fred_CC['value'])


In [3]:
# Reshape the data into quarterly columns
Fred_CC = pd.DataFrame(Fred_CC.values.reshape(-1,4), columns=['Q1_CH', 'Q2_CH', 'Q3_CH', 'Q4_CH'])

In [4]:
Fred_CC.head()

,Q1_CH,Q2_CH,Q3_CH,Q4_CH
0,634.935,636.563,638.068,639.683
1,641.058,642.074,644.160,646.535
2,647.333,649.092,651.362,653.524
3,654.259,656.944,658.904,660.584
4,662.441,664.260,665.773,667.762


In [5]:
Fred_CC.shape

(30, 4)

In [6]:
# Import bls library and connect to datasource, return as pd dataframe
import bls
df = bls.get_series('LEU0252881500', 2013, 2022)
df = pd.DataFrame({'date':df.index, 'value':df.values})

In [7]:
# Set date col as index and check info 
df.set_index('date', inplace = True)

df.info()

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 39 entries, 2013Q1 to 2022Q3
Freq: Q-DEC
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   value   39 non-null     float64
dtypes: float64(1)
memory usage: 624.0 bytes


In [8]:
# Add new row to reshape in preperation for concat with FRED data
new_row = {'value': 0}
df = df.append(new_row, ignore_index=True)
df = pd.DataFrame(df.values.reshape(-1, 4),columns=['Q1_Wage','Q2_Wage','Q3_Wage','Q4_Wage'])

In [9]:
#Concatenate previous dataframes
df1 = pd.concat([Fred_CC, df], axis=1, join='inner')

#Ensure dataframe created successfully, check lenth, datatypes, and nulls in the data
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Q1_CH    10 non-null     float64
 1   Q2_CH    10 non-null     float64
 2   Q3_CH    10 non-null     float64
 3   Q4_CH    10 non-null     float64
 4   Q1_Wage  10 non-null     float64
 5   Q2_Wage  10 non-null     float64
 6   Q3_Wage  10 non-null     float64
 7   Q4_Wage  10 non-null     float64
dtypes: float64(8)
memory usage: 768.0 bytes


In [10]:
df1.tail()

,Q1_CH,Q2_CH,Q3_CH,Q4_CH,Q1_Wage,Q2_Wage,Q3_Wage,Q4_Wage
5,668.789,670.744,672.722,674.253,881.0,876.0,887.0,900.0
6,677.508,678.628,681.693,684.516,905.0,908.0,919.0,936.0
7,686.271,688.241,690.594,691.881,957.0,1002.0,994.0,984.0
8,694.559,695.652,697.440,698.556,989.0,990.0,1001.0,1010.0
9,699.997,701.688,703.289,704.842,1037.0,1041.0,1070.0,0.0


In [32]:
#Check proportion of income allocated towards childcare, Q1 of each year
df1.apply(lambda x: x['Q1_Wage'] / x['Q1_CH'], axis=1)

0    1.217447
1    1.241697
2    1.248198
3    1.268611
4    1.305777
5    1.317306
6    1.335778
7    1.394493
8    1.423925
9    1.481435
dtype: float64

In [12]:
# Use LR model to get projected value for Q4, 2022
X = df1[['Q1_Wage','Q2_Wage','Q3_Wage']]
y = df1['Q4_Wage']
# Create training data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=24)
# Define model type (Linear Regression) and fit model on data 
lr = LinearRegression()
lr.fit(X_train, y_train);
# Print scores for training and test data
print(lr.score(X_train, y_train))

print(lr.score(X_test, y_test))
# Create predictions from LR model 
predictions = lr.predict(X)

print(predictions)
# Check residuals
residuals = y - predictions
residuals

0.9954724125681401
-0.9371853064093876
[ 781.91620416  805.41335467  823.94926346  849.57167688  896.87009104
  903.07044111  927.72060748  987.35845224 1033.98160698 1070.4733981 ]


0       4.083796
1      -6.413355
2       1.050737
3      -0.571677
4     -39.870091
5      -3.070441
6       8.279393
7      -3.358452
8     -23.981607
9   -1070.473398
Name: Q4_Wage, dtype: float64

In [13]:
# Replace Q4, 2022 data with model projection, ensure data has been replaced successfully
df1['Q4_Wage'] = np.where(df1['Q4_Wage'] == 0, 1070, df1['Q4_Wage'])
df1.tail()

,Q1_CH,Q2_CH,Q3_CH,Q4_CH,Q1_Wage,Q2_Wage,Q3_Wage,Q4_Wage
5,668.789,670.744,672.722,674.253,881.0,876.0,887.0,900.0
6,677.508,678.628,681.693,684.516,905.0,908.0,919.0,936.0
7,686.271,688.241,690.594,691.881,957.0,1002.0,994.0,984.0
8,694.559,695.652,697.440,698.556,989.0,990.0,1001.0,1010.0
9,699.997,701.688,703.289,704.842,1037.0,1041.0,1070.0,1070.0


In [33]:
# Use LR model to get projected value for Q4_CH (Childcare costs, 2022)
# Define new X and y variables
X1 = df1[['Q1_CH','Q2_CH','Q3_CH']]
y1 = df1['Q4_CH']

# Create training data
X_train, X_test, y_train, y_test = train_test_split(X1, y1, random_state=24)
# Fit model
lr.fit(X_train, y_train);
# Score model on train and test data

print(lr.score(X_train, y_train))

print(lr.score(X_test, y_test))

# Create, print predictions 
predictions = lr.predict(X1)

predictions

0.9999615993310399
0.9959922007582818


array([639.70516328, 646.50941592, 653.67972576, 660.49825422,
       666.72645465, 674.10857651, 684.42756159, 692.04730271,
       698.09737352, 703.4731571 ])

In [25]:
# Create dictionary to map quarters from income and childcare
dict = {'Q1_Wage':'Q1_CH', 'Q2_Wage':'Q2_CH', 'Q3_Wage':'Q3_CH', 'Q4_Wage':'Q4_CH'}

In [29]:
# Divide each income record with mapped childcare value, create new corresponding column

# Iterate through the columns of the DataFrame
for col in df1.columns:
    # Check if the column is in the dictionary
    if col in dict.keys():
        # Divide the column by the corresponding value in the dictionary
        df1[col+'_Ratio'] = df1[col] / df1[dict[col]]

In [30]:
df1

,Q1_CH,Q2_CH,Q3_CH,Q4_CH,Q1_Wage,Q2_Wage,Q3_Wage,Q4_Wage,Q1_Wage_Ratio,Q2_Wage_Ratio,Q3_Wage_Ratio,Q4_Wage_Ratio
0,634.935,636.563,638.068,639.683,773.0,776.0,771.0,786.0,1.217447,1.219047,1.208335,1.228734
1,641.058,642.074,644.160,646.535,796.0,780.0,790.0,799.0,1.241697,1.214813,1.226403,1.235819
2,647.333,649.092,651.362,653.524,808.0,801.0,803.0,825.0,1.248198,1.234032,1.232801,1.262387
3,654.259,656.944,658.904,660.584,830.0,824.0,827.0,849.0,1.268611,1.254293,1.255115,1.285226
4,662.441,664.260,665.773,667.762,865.0,859.0,859.0,857.0,1.305777,1.293168,1.290230,1.283391
5,668.789,670.744,672.722,674.253,881.0,876.0,887.0,900.0,1.317306,1.306012,1.318524,1.334811
6,677.508,678.628,681.693,684.516,905.0,908.0,919.0,936.0,1.335778,1.337994,1.348114,1.367390
7,686.271,688.241,690.594,691.881,957.0,1002.0,994.0,984.0,1.394493,1.455885,1.439341,1.422210
8,694.559,695.652,697.440,698.556,989.0,990.0,1001.0,1010.0,1.423925,1.423125,1.435249,1.445840
9,699.997,701.688,703.289,704.842,1037.0,1041.0,1070.0,1070.0,1.481435,1.483565,1.521423,1.518071
